In [1]:
import pandas as pd
from clickhouse_driver import Client

In [2]:
client_read_format = Client(
    host='host',
    user='komisarenko.aa',
    password='password'
)

In [3]:
q_1 = f'''

SELECT 
	SysRowDate,
	DivisionName,
	RDCName,
	BranchName,
	TerminalType,
	TerminalId,
	count() "Количество КЗ",
	sum(if(has(EventDescription, 'Добавлен товар (Каталог)'), 1, 0)) "Добавления через каталог", --1
	sum(if(has(EventDescription, 'Добавлен товар (Сканирование)'), 1, 0)) "Добавления через скан", --2
	sum(if(has(EventDescription, 'Добавлен товар (Сканирование)')
				AND has(EventDescription, 'Добавлен товар (Каталог)'), 1, 0)) "Добавления товара", --3
	sum(if(NOT has(EventDescription, 'Добавлен товар (Сканирование)')
				AND NOT has(EventDescription, 'Добавлен товар (Каталог)'), 1, 0)) "Оформления заказов", --4
	sum(if(has(EventDescription, 'Выбрана оплата на кассе'), 1, 0)) "Выбрана оплата на кассе", --5.1
	sum(if(has(EventDescription, 'Выбрана оплата по СБП'), 1, 0)) "Выбрана оплата по СБП", --5.2
	sum(if(has(EventDescription, 'Выбрана оплата банковской картой'), 1, 0)) "Выбрана оплата банковской картой", --5.3
	sum(if(has(EventDescription, 'Выбрана оплата на кассе')
            OR has(EventDescription, 'Выбрана оплата по СБП')
            OR has(EventDescription, 'Выбрана оплата банковской картой'), 1, 0)) "Переходы на оплату", --5
    sum(if(has(EventDescription, 'Заказ оплачен') 
            OR has(EventDescription, 'Заказ оплачен СБП'), 1, 0)) "Оплаты в Автокассе", --6
	sum(if(NOT has(EventDescription, 'Заказ оплачен') 
			AND has(EventDescription, 'Выбрана оплата банковской картой')
            OR NOT has(EventDescription, 'Заказ оплачен СБП') 
            AND has(EventDescription, 'Выбрана оплата по СБП'), 1, 0)) "Незавершенные оплаты", --7
    sum(if((has(EventDescription, 'Добавлен товар (Сканирование)') 
            OR has(EventDescription, 'Добавлен товар (Каталог)'))
            AND (has(EventDescription, 'Заказ оплачен')
            OR has(EventDescription, 'Заказ оплачен СБП')), 1, 0)) "Чистая оплата", --8
	sum(if((has(EventDescription, 'Добавлен товар (Сканирование)') 
            OR has(EventDescription, 'Добавлен товар (Каталог)'))
            AND NOT has(EventDescription, 'Оформлен заказ'), 1, 0)) "Брошенные корзины" --9
FROM (
		SELECT 
			SysRowDate,
			DivisionName,
			RDCName,
			BranchName,
			TerminalType,
			TerminalId,
			OrderGuid,
			groupArray(EventDescription) EventDescription
		FROM (
			SELECT
				*
			FROM (
				SELECT 
					SysRowDate,
					DivisionName,
					RDCName,
					BranchName,
					OrderGuid,
					if(
						EventSystemInterface = 'catalog', 'Автокасса с каталогом',
						'Автокасса'
						) TerminalType,
					multiIf(
							EventTitle = 'Сканирование' AND EventDescription = 'Добавлен товар', 'Добавлен товар (Сканирование)',
							EventTitle = 'Нажатие кнопки' AND EventDescription = 'Добавлен товар', 'Добавлен товар (Каталог)',
							EventDescription
							) EventDescription
				FROM SalesEventsLog
				WHERE toStartOfYear(SysRowDate) = '2024-01-01'
					AND EventSystemType IN ('Автокасса', 'Платежный шлюз')
					AND EventTitle <> ''
					AND EventDescription <> ''
					AND BranchName <> '') sel1
				JOIN (
					SELECT DISTINCT
						OrderGuid,
						JSONExtractString(OrderObject, 'selfServiceTerminalId') AS TerminalId
					FROM SalesEventsLog
					WHERE toStartOfYear(SysRowDate) = '2024-01-01'
						AND TerminalId <> ''
						AND EventSystemType IN ('Автокасса', 'Платежный шлюз')
						AND EventTitle <> ''
						AND EventDescription <> ''
						AND BranchName <> '') sel2
				USING OrderGuid
		)
		GROUP BY 
			SysRowDate,
			DivisionName,
			RDCName,
			BranchName,
			TerminalType,
			TerminalId,
			OrderGuid)
GROUP BY 
	SysRowDate,
	DivisionName,
	RDCName,
	BranchName,
	TerminalType,
	TerminalId;
    
'''

req1, col1 = client_read_format.execute(q_1, with_column_types=True)
df_1 = pd.DataFrame(req1, columns=[x[0] for x in col1])

In [25]:
#df_1.head(2)

In [5]:
q_2 = f'''

SELECT DISTINCT 
	SysRowDate,
    DivisionName,
    RDCName,
    BranchName,
    TerminalType,
    TerminalId,
    System,
    arrayJoin(arrayFilter(x -> x <> 'Заказ оплачен на Автокассе', EventDescription)) AS EventDescription,
    arrayJoin(time_differences) "Время до оплаты товара после добавления"
FROM (
	WITH 
	    arrayFilter((x, y) -> y IN ('Добавлен товар (Сканирование)', 'Добавлен товар (Каталог)'), EventDateUTC, EventDescription) AS add_tovar_events,
	    arrayFilter((x, y) -> y = 'Заказ оплачен на Автокассе', EventDateUTC, EventDescription) AS oplata_events
	SELECT 
	    SysRowDate,
	    DivisionName,
	    RDCName,
	    BranchName,
	    TerminalType,
	    System,
	    TerminalId,
	    OrderGuid,
	    groupArray(EventDescription) AS EventDescription,
	    groupArray(EventDateUTC) AS EventDateUTC,
	    arrayJoin(arrayMap((add_event) ->
	        arrayMap((oplata_event) -> 
	            dateDiff('second', add_event, oplata_event), 
	            oplata_events), 
	        add_tovar_events)) AS time_differences
	FROM (
		SELECT
			*
		FROM (
			SELECT  
		        SysRowDate,
		        EventDateUTC,
		        DivisionName,
		        RDCName,
		        BranchName,
		        OrderGuid,
		        if(
					EventSystemInterface = 'catalog', 'Автокасса с каталогом',
					'Автокасса'
					) TerminalType,
		        multiIf(
		            EventTitle = 'Сканирование' AND EventDescription = 'Добавлен товар', 'Добавлен товар (Сканирование)',
		            EventTitle = 'Нажатие кнопки' AND EventDescription = 'Добавлен товар', 'Добавлен товар (Каталог)',
		            EventDescription = 'Заказ оплачен' OR EventDescription = 'Заказ оплачен СБП', 'Заказ оплачен на Автокассе',
		            'Другое') AS EventDescription,
		        if(
		            UserName <> 'Автокасса', 'Автокасса+сотрудник',
		            'Автокасса') AS System
		    FROM SalesEventsLog
		    WHERE toStartOfYear(SysRowDate) = '2024-01-01'
		        AND EventSystemType IN ('Автокасса', 'Платежный шлюз')
		        AND EventTitle <> ''
		        AND EventDescription <> ''
		        AND BranchName <> ''
		    HAVING EventDescription <> 'Другое') sal1
		JOIN (
			SELECT DISTINCT
				OrderGuid,
				JSONExtractString(OrderObject, 'selfServiceTerminalId') AS TerminalId
			FROM SalesEventsLog
			WHERE toStartOfYear(SysRowDate) = '2024-01-01'
				AND TerminalId <> ''
				AND EventSystemType IN ('Автокасса', 'Платежный шлюз')
				AND EventTitle <> ''
				AND EventDescription <> ''
				AND BranchName <> '') sel2
		USING OrderGuid
	)
	GROUP BY 
	    SysRowDate,
	    DivisionName,
	    RDCName,
	    BranchName,
	    TerminalType,
	    System,
	    TerminalId,
	    OrderGuid
	HAVING arrayCount(x -> x = 'Добавлен товар (Сканирование)' OR x = 'Добавлен товар (Каталог)', EventDescription) >= 1
	    AND arrayCount(x -> x = 'Заказ оплачен на Автокассе', EventDescription) >= 1);
    
'''

req1, col1 = client_read_format.execute(q_2, with_column_types=True)
df_2 = pd.DataFrame(req1, columns=[x[0] for x in col1])

In [23]:
#df_1.head(2)

In [7]:
#df_1.to_csv('sda_111_1.csv', index=False)

In [8]:
#df_2.to_csv('sda_111_2.csv', index=False)